# LP: Záměrně neomezená úloha v PuLP

Cíl: ukázat triviální LP, kde hodnota cíle roste do nekonečna. Solver má vrátit stav *Unbounded*.

**Model**
- Proměnná: $x \ge 0$
- Maximalizace: $\max x$
- Omezení: žádná horní mez na $x$ ⟶ účelová funkce roste libovolně ⟶ neomezené

Postup:
1) Instalace a import PuLP  
2) Sestavení modelu  
3) Řešení s výpisem logu solveru  
4) Kontrola stavu, hodnot proměnných a cíle  
5) Uložení modelu do `unbounded.lp` pro inspekci


In [ ]:
# 1) Instalace a import PuLP
try:
    import pulp
except ImportError:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pulp"])
    import pulp

print("PuLP verze:", pulp.__version__)
print("Dostupní řešitelé:", pulp.listSolvers(onlyAvailable=True))


PuLP verze: 3.3.0
Dostupní řešitelé: ['PULP_CBC_CMD', 'HiGHS']


## Sestavení neomezené LP úlohy

Formulace:
- jedna proměnná $x \ge 0$
- účelová funkce $\max x$
- bez omezení, která by omezila horní mez $x$

Očekávaný výsledek: solver vrátí `Unbounded`.


In [ ]:
# 2) Formulace modelu v PuLP
from pulp import LpProblem, LpMaximize, LpVariable, LpStatus, value, PULP_CBC_CMD

prob = LpProblem("unbounded_demo", LpMaximize)

# x >= 0, žádná horní mez
x = LpVariable("x", lowBound=0, upBound=None, cat="Continuous")

# Cíl: max x
prob += x, "objective"

# Volitelně: uložit LP do souboru
prob.writeLP("unbounded.lp")
print("Model uložen do: unbounded.lp")


Model uložen do: unbounded.lp


## Řešení a výpis logu solveru

Spustíme CBC s `msg=True`, aby se log vypsal do notebooku. CBC by měl detekovat neomezenou účelovou funkci.


In [ ]:
# 3) Řešení s CBC, zapnutý log
solver = PULP_CBC_CMD(msg=True)
status_code = prob.solve(solver)

print("\n--- Souhrn stavu ---")
print("Status kód:", status_code)
print("Status text:", LpStatus[prob.status])

# 4) Kontrola hodnot proměnných a cíle
def safe_val(v):
    return None if v.varValue is None else float(v.varValue)

obj_val = None
try:
    obj_val = value(prob.objective)
except Exception:
    obj_val = None

print("x =", safe_val(x))
print("Hodnota cíle =", obj_val)



--- Souhrn stavu ---
Status kód: -2
Status text: Unbounded
x = 0.0
Hodnota cíle = 0.0


## Co sledovat ve výstupu

- Log CBC by měl indikovat neomezenou úlohu.
- `LpStatus` vrátí `Unbounded`.

Rychlý test:
- Přidejte horní mez, např. omezení `x <= 10`, a stav se změní na `Optimal` s hodnotou účelové funkce 10.
